In [1]:
sensitive_attributes = {'sex': {'columns': [
                                            {
                                                "name": "sex",
                                                "underprivileged": [2]
                                            }
                                        ],
                                'type': 'simple'},
                        'ethnicity': {'columns': [
                                                 {
                                                     "name": "ethnicity",
                                                     "privileged": [1]
                                                 }
                                        ],
                                      'type': 'simple'},
                        'age': {'columns': [
                                            {
                                                 "name": "age",
                                                 "privileged": [3, 4]
                                            }
                                        ],
                                'type': 'simple'},
                        'error': {'columns': [
                            {
                                "name": "error",
                                  "privileged": [3, 4]
                                  }
                                  ],
                                  'type': 'simple'},
                        'sex_ethnicity': {'groups': [
                                                    "sex", "ethnicity"
                                                    ],
                                          'type': 'complex'}}

import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s - %(message)s'
)

from eticas.model.ml_model import MLModel
model = MLModel(
    model_name="ML Testing Regression",
    description="A logistic regression model to illustrate audits",
    country="USA",
    state="CA",
    sensitive_attributes=sensitive_attributes,
    features=["feature_0", "feature_1", "feature_2"]
)


model.run_labeled_audit(dataset_path ='files/example_training_binary_2.csv', 
                                                label_column = 'outcome', output_column = 'predicted_outcome',positive_output = [1])
model.run_production_audit(dataset_path ='files/example_operational_binary_2.csv', 
                                                output_column = 'predicted_outcome',positive_output = [1])
model.run_impacted_audit(dataset_path ='files/example_impact_binary_2.csv', 
                                                output_column = 'recorded_outcome',positive_output = [1])
model.run_drift_audit(dataset_path_dev = 'files/example_training_binary_2.csv', 
                      output_column_dev = 'outcome', positive_output_dev = [1],
                    dataset_path_prod = 'files/example_operational_binary_2.csv', 
                      output_column_prod = 'predicted_outcome', positive_output_prod = [1])


[INFO] eticas.model.ml_model - Running labeled audit for model: ML Testing Regression
[INFO] eticas.audit.labeled_audit - Running labeled audit on model 'ML Testing Regression'
[INFO] eticas.data.loaders - Dataset successfully loaded from files/example_training_binary_2.csv (CSV).
[INFO] eticas.audit.labeled_audit - labeled data loaded '(10000, 10)'
[ERROR] eticas.metrics.da_inconsistency - Group no present in data: 'error'
[INFO] eticas.metrics.da_inconsistency - Completed: 'Da_inconsistency'
[ERROR] eticas.metrics.da_positive - Group no present in data: 'error'
[INFO] eticas.metrics.da_positive - Completed: 'Da_positive'
[ERROR] eticas.metrics.dxa_inconsistency - Group no present in data: 'error'
[INFO] eticas.metrics.dxa_inconsistency - Completed: 'Dxa_inconsistency'
[ERROR] eticas.metrics.da_informative - Group no present in data: 'error'
[INFO] eticas.metrics.da_informative - Completed: 'Da_informative'
[ERROR] eticas.metrics.da_fairness - Group no present in data: 'error'
[INFO] 

In [7]:
import pandas as pd
pd.set_option('display.max_rows', 500)

audit_result = model.df_results(norm_values=True)

In [7]:
audit_result.reset_index().to_pickle('tests/ml_files/result_df.pickle')

In [10]:
import json
with open('tests/ml_files/result_json_production.json', "w") as file:
    json.dump(model.production_results, file, indent=4)

In [8]:
result = audit_result.xs(('fairness'), level=(0))
result = result.reset_index()
result = result.pivot(index=['metric','attribute'], 
                      columns='stage', 
                      values='value')
result

stage                             01-labeled 02-production 03-impact
metric              attribute                                       
d_equality          age                 99.0          99.0     100.0
                    ethnicity          100.0          99.0      44.0
                    sex                100.0          71.0      47.0
                    sex_ethnicity       99.0          62.0      44.0
d_equity            age                100.0          99.0     100.0
                    ethnicity          100.0          70.0      44.0
                    sex                 99.0          48.0      41.0
                    sex_ethnicity      100.0          57.0      44.0
d_parity            age                 98.0          98.0      99.0
                    ethnicity           99.0     76.666667      42.0
                    sex                 99.0     64.666667      40.8
                    sex_ethnicity       99.0     65.333333      40.8
d_statisticalparity age                 98.0          98.0      98.0
                    ethnicity          100.0          74.0      43.5
                    sex                100.0          57.0      42.0
                    sex_ethnicity       98.0         57.75      42.0

In [8]:
result = audit_result.xs(('performance'), level=(0))
result = result.reset_index()
result = result.pivot(index=['metric','attribute'], 
                      columns='stage', 
                      values='value')
result

stage                          01-labeled
metric           attribute               
FN               age                 1151
                 ethnicity            987
                 sex                 1475
                 sex_ethnicity        619
FP               age                 1146
                 ethnicity           1027
                 sex                 1565
                 sex_ethnicity        653
TN               age                 1060
                 ethnicity            973
                 sex                 1428
                 sex_ethnicity        605
TP               age                 1121
                 ethnicity           1013
                 sex                 1532
                 sex_ethnicity        623
accuracy         age                 48.7
                 ethnicity          49.65
                 sex                49.33
                 sex_ethnicity      49.12
f1               age                49.39
                 ethnicity          50.15
                 sex                 50.2
                 sex_ethnicity      49.48
poor_performance age               57.588
                 ethnicity          59.58
                 sex               59.052
                 sex_ethnicity     58.572
precision        age                49.45
                 ethnicity          49.66
                 sex                49.47
                 sex_ethnicity      48.82
recall           age                49.34
                 ethnicity          50.65
                 sex                50.95
                 sex_ethnicity      50.16

In [12]:
audit_result.reset_index().metric.unique()

array(['da_inconsistency', 'da_positive', 'da_informative',
       'dxa_inconsistency', 'drift', 'd_equality', 'd_equity', 'd_parity',
       'd_statisticalparity', 'd_calibrated_false', 'd_calibrated_true',
       'd_equalodds_false', 'd_equalodds_true', 'FN', 'FP', 'TN', 'TP',
       'accuracy', 'f1', 'poor_performance', 'precision', 'recall'],
      dtype=object)

In [13]:
reshaped_df = (
    audit_result.xs(('benchmarking'), level=(0)).reset_index()
    .pivot(index=[ 'metric', 'attribute'], columns='stage', values='value')
    .reset_index()  # Convierte los índices de nuevo a columnas
)
reshaped_df.columns.name = None
reshaped_df

,metric,attribute,01-labeled,02-production,03-impact
0,da_inconsistency,age,44.8,44.5,45.0
1,da_inconsistency,ethnicity,40.0,20.0,10.0
2,da_inconsistency,sex,60.0,30.0,15.0
3,da_inconsistency,sex_ethnicity,25.0,15.0,10.0
4,da_positive,age,45.2,44.9,45.3
5,da_positive,ethnicity,39.8,15.8,3.7
6,da_positive,sex,59.8,19.7,5.7
7,da_positive,sex_ethnicity,24.7,10.0,3.7


In [19]:
model.labeled_results['d_equalodds']['sex']

{'true_positive_rate': {'underprivileged': 0.5095,
  'privileged': 0.4916,
  'ratio_true': 1.0364,
  'normalized_risk': 96.36,
  'bias_level': 'Low/no bias'},
 'false_positive_rate': {'underprivileged': 0.5229,
  'privileged': 0.5161,
  'ratio_false': 1.0131,
  'normalized_risk': 98.69,
  'bias_level': 'Low/no bias'}}

In [21]:
model.labeled_results['d_calibrated']['sex']

{'true_calibrated': {'underprivileged': 0.5095,
  'privileged': 0.4916,
  'ratio_true': 1.0364,
  'normalized_risk': 96.36,
  'bias_level': 'Low/no bias'},
 'false_calibrated': {'underprivileged': 0.5229,
  'privileged': 0.5161,
  'ratio_false': 1.0131,
  'normalized_risk': 98.69,
  'bias_level': 'Low/no bias'}}